<a href="https://colab.research.google.com/github/upadhyayprakash/eva4/blob/master/S4/EVA4_Session_4_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Task
Train MNIST to **99.4%** or more validation accuracy with following constraints:
1. Less than 20k parameters
2. less than 20 Epochs
3. No Fully Connected Layer(almost)
4. Try all the concepts taught in the class

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
# Old Architecture, Don't use this
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        print('#### OLD ####')
        print(x.shape)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [0]:
class NewNet(nn.Module):
    def __init__(self):
        super(NewNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #input - 28x28x1 | Kernel Size - 3x3x1 | Output - 28x28x32 | RF = 3
        self.pool1 = nn.MaxPool2d(2, 2) #input - 28x28x64 | Output - 14x14x64 | RF = 10
        self.conv2 = nn.Conv2d(16, 32, 3, stride=2) # Newly Added
        self.conv4 = nn.Conv2d(16, 10, 3, padding=1) # Newly Added
        self.convOne1 = nn.Conv2d(32, 16, 1) # Newly Added
        
    def forward(self, x):
        x = self.pool1((F.relu(self.conv1(x))))
        x = F.relu(self.conv2(x))
        x = self.pool1(x)
        x = self.convOne1(x)
        # x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.convOne1(x)
        x = self.conv4(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [4]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = NewNet().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
         MaxPool2d-2           [-1, 16, 14, 14]               0
            Conv2d-3             [-1, 32, 6, 6]           4,640
         MaxPool2d-4             [-1, 32, 3, 3]               0
            Conv2d-5             [-1, 16, 3, 3]             528
            Conv2d-6             [-1, 32, 1, 1]           4,640
            Conv2d-7             [-1, 16, 1, 1]             528
            Conv2d-8             [-1, 10, 1, 1]           1,450
Total params: 11,946
Trainable params: 11,946
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.13
Params size (MB): 0.05
Estimated Total Size (MB): 0.18
----------------------------------------------------------------


<ipython-input-3-25ae6e1abecf>:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [0]:
torch.manual_seed(1)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/home/prakash/Prakash/EVA4/Session-4/Notebooks/dataMNIST', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/home/prakash/Prakash/EVA4/Session-4/Notebooks/dataMNIST', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [19]:
model = NewNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 5):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/1875 [00:00<?, ?it/s]<ipython-input-3-25ae6e1abecf>:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0907, Accuracy: 9713/10000 (97%)



  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0721, Accuracy: 9781/10000 (98%)



  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0669, Accuracy: 9782/10000 (98%)



loss=0.010820388793945312 batch_id=1874: 100%|██████████| 1875/1875 [00:11<00:00, 159.61it/s]



Test set: Average loss: 0.0627, Accuracy: 9805/10000 (98%)



In [8]:
torch.device('cuda')

device(type='cuda')